Set up env and imports

In [1]:
!pip install nada-dsl==0.5.0
!pip install py-nillion-client==0.5.0
!pip install nillion-python-helpers==0.2.3
!pip install python-dotenv==1.0.0

In [2]:
import py_nillion_client as nillion
from py_nillion_client import NodeKey, UserKey
from dotenv import load_dotenv
from nillion_python_helpers import get_quote_and_pay, create_nillion_client, create_payments_config
from cosmpy.aerial.client import LedgerClient
from cosmpy.aerial.wallet import LocalWallet
from cosmpy.crypto.keypairs import PrivateKey

In [13]:
import os

load_dotenv('/home/vishakh/.config/nillion/nillion-devnet.env')

for key, value in os.environ.items():
    print(f"{key}: {value}")

PATH: /home/vishakh/dev/cryptonomic/MonadicDNA/services/nillion-interactor/venv/bin:/home/vishakh/.nvm/versions/node/v12.22.3/bin:/home/vishakh/.cargo/bin:/home/vishakh/.opam/default/bin:/home/vishakh/.local/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/snap/bin:/snap/bin:/home/vishakh/apps/go/bin
XAUTHORITY: /run/user/1000/gdm/Xauthority
INVOCATION_ID: 1878f8ab32de4ac3aa79aec5ef7ce9c8
XMODIFIERS: @im=ibus
XDG_DATA_DIRS: /usr/share/ubuntu:/usr/share/gnome:/usr/local/share/:/usr/share/:/var/lib/snapd/desktop
GDMSESSION: ubuntu
MANDATORY_PATH: /usr/share/gconf/ubuntu.mandatory.path
GTK_IM_MODULE: ibus
DBUS_SESSION_BUS_ADDRESS: unix:path=/run/user/1000/bus
DEFAULTS_PATH: /usr/share/gconf/ubuntu.default.path
PS1: (venv) 
XDG_CURRENT_DESKTOP: ubuntu:GNOME
JOURNAL_STREAM: 8:52068
SESSION_MANAGER: local/vishakhs-super-pc:@/tmp/.ICE-unix/4686,unix/vishakhs-super-pc:/tmp/.ICE-unix/4686
USERNAME: vishakh
LOGNAME: vishakh
PWD: /home/vishakh/dev/cryp

In [14]:
cluster_id = os.getenv('NILLION_CLUSTER_ID')
chain_id = os.getenv('NILLION_NILCHAIN_CHAIN_ID')
grpc_endpoint = os.getenv('NILLION_NILCHAIN_GRPC')

print(f"Cluster ID: {cluster_id}")
print(f"Chain ID: {chain_id}")
print(f"GRPC Endpoint: {grpc_endpoint}")

Cluster ID: 9e68173f-9c23-4acc-ba81-4f079b639964
Chain ID: nillion-chain-devnet
GRPC Endpoint: localhost:26649


In [15]:
import uuid
def gen_node_key():
    return NodeKey.from_seed(uuid.uuid4().hex)

In [16]:
monadic_seed = "monadic_seed"
monadic_userkey = UserKey.from_seed(monadic_seed)
monadic_client = create_nillion_client(monadic_userkey, gen_node_key())
monadic_party_id = monadic_client.party_id
monadic_user_id = monadic_client.user_id

In [17]:
snipper_seed = "snipper_seed"
snipper_userkey = UserKey.from_seed(snipper_seed)
snipper_client = create_nillion_client(snipper_userkey, gen_node_key())
snipper_party_id = snipper_client.party_id
snipper_user_id = snipper_client.user_id

In [18]:
seed = "my_seed"
userkey = UserKey.from_seed(seed)
client = create_nillion_client(userkey, gen_node_key())
party_id = client.party_id
user_id = client.user_id

In [19]:
payments_config = create_payments_config(chain_id, grpc_endpoint)
payments_client = LedgerClient(payments_config)
payments_wallet = LocalWallet(
    PrivateKey(bytes.fromhex(os.getenv("NILLION_NILCHAIN_PRIVATE_KEY_0"))),
    prefix="nillion",
)

In [20]:
program_name = "double"
program_mir_path = f"binaries/double.nada.bin"

In [21]:
async def store_program():
    receipt_store_program = await get_quote_and_pay(
        client,
        nillion.Operation.store_program(program_mir_path),
        payments_wallet,
        payments_client,
        cluster_id,
    )

    # Store the program
    action_id = await client.store_program(
        cluster_id, program_name, program_mir_path, receipt_store_program
    )
    return action_id

# Use this in your Jupyter notebook cell
action_id = await store_program()
print(f"Action ID: {action_id}")

Getting quote for operation...
Quote cost is 2 unil
Submitting payment receipt 2 unil, tx hash BF1E056995EA02FE9C804B0FFAF2A2A277C422A3B6BDA5C0B526DE6691103FA9
Action ID: 3rgqxWd47e171EUwe4RXP9hm45tmoXfuF8fC52S7jcFoQTnU8wPiL7hqWzyV1muak6bEg7iWhudwg4t2pM9XnXcp/double


In [22]:
program_id = f"{user_id}/{program_name}"

new_secret = nillion.NadaValues(
    {
        "foo": nillion.SecretInteger(2),
    }
)

party_name = "Party1"

# Set permissions for the client to compute on the program
permissions = nillion.Permissions.default_for_user(client.user_id)
permissions.add_compute_permissions({client.user_id: {program_id}})

# Pay for and store the secret in the network and print the returned store_id
receipt_store = await get_quote_and_pay(
    client,
    nillion.Operation.store_values(new_secret, ttl_days=5),
    payments_wallet,
    payments_client,
    cluster_id,
)
# Store a secret
store_id = await client.store_values(
    cluster_id, new_secret, permissions, receipt_store
)

Getting quote for operation...
Quote cost is 482 unil
Submitting payment receipt 482 unil, tx hash 40225AB14D082A3855740DBBCC3EDB771FA25338490BE8CE8286F56C24397056


In [23]:
compute_bindings = nillion.ProgramBindings(program_id)
compute_bindings.add_input_party(party_name, party_id)
compute_bindings.add_output_party(party_name, party_id)

computation_time_secrets = nillion.NadaValues({})

# Pay for the compute
receipt_compute = await get_quote_and_pay(
    client,
    nillion.Operation.compute(program_id, computation_time_secrets),
    payments_wallet,
    payments_client,
    cluster_id,
)

# Compute on the secret
compute_id = await client.compute(
    cluster_id,
    compute_bindings,
    [store_id],
    computation_time_secrets,
    receipt_compute,
)

# 8. Return the computation result
print(f"The computation was sent to the network. compute_id: {compute_id}")
while True:
    compute_event = await client.next_compute_event()
    if isinstance(compute_event, nillion.ComputeFinishedEvent):
        print(f"✅  Compute complete for compute_id {compute_event.uuid}")
        print(f"🖥️  The result is {compute_event.result.value}")
        break

Getting quote for operation...
Quote cost is 3 unil
Submitting payment receipt 3 unil, tx hash 9A79F70D6E43D9B348378D153E88B88850ECCE879381120149C9AEFDCCB68001
The computation was sent to the network. compute_id: 68da2ed9-5f5e-4e9a-8441-d37d3e62a5df
✅  Compute complete for compute_id 68da2ed9-5f5e-4e9a-8441-d37d3e62a5df
🖥️  The result is {'my_output': 4}
